In [1]:
import numpy as np
import scipy as sp
import copy as copy
import pandas as pd
import time
import copy
from npl import bootstrap_gmm as bgmm
from npl.maximise_gmm import init_toy
from npl.maximise_gmm import sampleprior_toy
from joblib import Parallel, delayed
from tqdm import tqdm
from npl import maximise_gmm as mgmm

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

import importlib
from npl.evaluate import gmm_ll as gll

/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/Users/cuifuheng/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
n_iter=1 #Times of experiments

def main(seed, modes):
    np.random.seed(seed)

    #Define GMM parameters
    K = 3        # number of clusters
    D = 1        ###dimension
    a=1 #hyperparameter for Dirichlet distr'n for Pi
    #Pi = np.random.dirichlet((a,a,a))
    Pi = np.array([[1/6,2/6,3/6],[1/3,1/3,1/3]])
    zvalues = np.array([0,1,2])
  
    if modes == 'sep':
        #MEANvalues = np.array([[0,0],
        #                       [2,2],
        #                       [4,4]])                   ### mean values
        MEANvalues = np.array([[0,2,4]]).T
    elif modes == 'insep':
        #MEANvalues = np.array([[0,0],
        #                       [1,1],
        #                       [2,2]])                   ### mean values
        MEANvalues = np.array([[0,1,2]]).T
    #sigvalues = np.array([[[1,0],
        #                   [0,1]],
        #                  [[1,0],
        #                   [0,1]],
        #                  [[1,0],
        #                   [0,1]]]) # deviance used in sampling layer
    
    sigvalues=np.array([1,1,1])
    
    #SIG=np.array([[1,0],
    #              [0,1]]) # deviance used in top layer (sampling mu_k_set)
    #SIG=np.array([1,1])

    SIG=0.3
    
    #Sample training z and y
    #set two different datasets i.e. K_set=2
    #row: set; columns: different indicators or variables
    
    K_set=2
    meanvalues=np.zeros((K_set,K,D))
    
    
    N = 10
    z = np.zeros((K_set,N),dtype = 'int')
    y = np.zeros((K_set,N,D))
    
    #import pdb
    #pdb.set_trace()
    #################################
    for i in range(K_set):
        meanvalues[i]=MEANvalues
        #meanvalues[i]=np.array([np.random.normal(MEANvalues[0],SIG),np.random.normal(MEANvalues[1],SIG),\
        #                        np.random.normal(MEANvalues[2],SIG)]).reshape((K,D))
        
        
        
        zrng = np.random.rand(K_set*N).reshape((K_set,N))
        yrng = np.random.randn(K_set*N*D).reshape((K_set,N,D))
        
        for j in range(N):
            ind = np.random.multinomial(1, Pi[i]).argmax()
            z[i,j]= zvalues[ind]
            if D==1:
                y[i,j,] = sigvalues[ind]*yrng[i,j,] + meanvalues[i,ind,]
            else:
                y[i,j,] = sigvalues[ind]@yrng[i,j,] + meanvalues[i,ind,]
            
            
        #y = y.reshape(N,D)

    #Concatenate and save training
    gmm_data = {'y': y, 'z': z, 'N':N,'K': K,'D':D, 'Pi' : Pi, 'zvals': zvalues, 'meanvals': meanvalues, 'sigvals':sigvalues, 'a':a, 'MEANvalues':MEANvalues, 'SIG':SIG, 'K_set':K_set}
    np.save('./sim_data/gmm_data_{}_seed{}'.format(modes,seed),gmm_data)


    #Sample test z and y
    
    N_test = 1000
    z_test = np.zeros((K_set,N_test),dtype = 'int')
    y_test = np.zeros((K_set,N_test,D))
    for i in range(K_set):
        meanvalues[i]=np.array([np.random.normal(MEANvalues[0],SIG),np.random.normal(MEANvalues[1],SIG),\
                                np.random.normal(MEANvalues[2],SIG)]).reshape((K,D))
        
        
        zrng = np.random.rand(K_set*N_test).reshape((K_set,N_test))
        yrng = np.random.randn(K_set*N_test*D).reshape((K_set,N_test,D))
        
        for j in range(N_test):
            ind = np.random.multinomial(1, Pi).argmax()
            z_test[i,j]= zvalues[ind]
            if D==1:
                y_test[i,j,] = sigvalues[ind]*yrng[i,j,] + meanvalues[i,ind,]
            else:
                y_test[i,j,] = sigvalues[ind]@yrng[i,j,] + meanvalues[i,ind,]
            
            
        #y_test = y_test.reshape(N_test,D)

    

    #Concatenate and save test data
    gmm_data_test = {'y': y_test, 'z': z_test, 'N':N_test,'K': K,'D':D, 'Pi' : Pi, 'zvals': zvalues, 'meanvals': meanvalues, 'sigvals':sigvalues, 'a':a, 'MEANvalues':MEANvalues, 'SIG':SIG, 'K_set':K_set}
    np.save('./sim_data/gmm_data_test_{}_seed{}'.format(modes,seed),gmm_data_test)

for i in range(n_iter):
    seed = 100+i
    main(seed,'sep')

In [3]:
def load_data(seed,K_set):
    #load data and parameters
    gmm_data = np.load('./sim_data/gmm_data_sep_seed{}.npy'.format(seed),allow_pickle = True).item()

    #Extract parameters from data
    N_data = gmm_data['N']
    K_clusters = gmm_data['K']
    D_data = gmm_data['D']
    y = gmm_data['y']
    
    return y, N_data, K_clusters, D_data, K_set

In [4]:
def sample_weights_and_prior(B_postsamples, alph_conc, N_data, D_data, K_clusters, T_trunc=100, postsamples= None):
    K_set=2
    alpha_top_layer=1
    eps_dirichlet=10**(-100)
    N_data=N_data*K_set
    if alph_conc!=0: #gamma
        alphas = np.concatenate((np.ones(N_data), (alph_conc/T_trunc)*np.ones(T_trunc)))
        beta_weights = np.random.dirichlet(alphas,B_postsamples)
        y_prior = sampleprior_toy(D_data,T_trunc,K_clusters,B_postsamples, postsamples)
        alpha_top_layer_beta=alpha_top_layer*beta_weights
        Weights_1=np.repeat([np.concatenate((np.ones(int(N_data/2)), np.zeros(int(N_data/2+T_trunc))))],B_postsamples,axis=0)+alpha_top_layer_beta
        Weights_2=np.repeat([np.concatenate((np.zeros(int(N_data/2)),np.ones(int(N_data/2)), np.zeros(T_trunc)))],B_postsamples,axis=0)+alpha_top_layer_beta
    else:
        beta_weights = np.random.dirichlet(np.ones(N_data), B_postsamples)
        y_prior = np.zeros(B_postsamples)
        alpha_top_layer_beta=alpha_top_layer*beta_weights
        Weights_1=np.repeat([np.concatenate((np.ones(int(N_data/2)),\
                                             np.zeros(int(N_data/2))))],B_postsamples,axis=0)+alpha_top_layer_beta
        Weights_2=np.repeat([np.concatenate((np.zeros(int(N_data/2)),np.ones(int(N_data/2))))]\
                            ,B_postsamples,axis=0)+alpha_top_layer_beta
    
    
    
    weights_1=Weights_1
    weights_2=Weights_2
    
    for b in range(B_postsamples):
        weights_1[b,:]=np.random.dirichlet(Weights_1[b,:]+eps_dirichlet,1)
        weights_2[b,:]=np.random.dirichlet(Weights_2[b,:]+eps_dirichlet,1)
        
    return weights_1, weights_2, y_prior

In [5]:
def hdp_gmm_joint(y,y_prior,weights_1,weights_2,N_data,K_clusters,D_data,alph_conc=0,tol=10**(-7),max_iter=100):
    
    pi_init_1,mu_init,sigma_init = mgmm.init_params(y[0],N_data,K_clusters,D_data,tol,max_iter)
    pi_init_2,_,_ = mgmm.init_params(y[1],N_data,K_clusters,D_data,tol,max_iter)
    
    pi_1 = pi_init_1
    pi_2 = pi_init_2
    mu = mu_init
    #mu=np.array([0,2,4])
    #sigma = np.array([np.diag(sigma_init[0]),np.diag(sigma_init[1]),np.diag(sigma_init[2])])
    sigma=np.array([1,1,1])
    
    for i in range(max_iter):
        eta_1 = E_STEP(y, y_prior, pi_1, mu, sigma, K_clusters, alph_conc)
        eta_2 = E_STEP(y, y_prior, pi_2, mu, sigma, K_clusters, alph_conc)
        
        pi_1 = M_STEP_pi(weights_1, eta_1)
        pi_2 = M_STEP_pi(weights_2, eta_2)
        
        mu = M_STEP_mu(y,y_prior,eta_1,eta_2,weights_1,weights_2, alph_conc, K_clusters, D_data)
        
        sigma = M_STEP_sigma(y,y_prior,eta_1,eta_2,weights_1,weights_2, alph_conc, K_clusters, D_data, mu)
        
    return pi_1, pi_2, mu, sigma

In [6]:
def E_STEP(y, y_prior, pi, mu, sigma, K_clusters, alph_conc):
    
    #print(pi.shape)
    
    if alph_conc!=0:
        y_con=np.concatenate((y[0],y[1],y_prior),axis=0)
    else:
        y_con=np.concatenate((y[0],y[1]),axis=0)
        
    N_data_tol=y_con.shape[0]
    eta=np.zeros((N_data_tol,K_clusters))
    
    for i in range(N_data_tol):
        for j in range(K_clusters):
            eta[i,j] = ((pi[j])*sp.stats.multivariate_normal(mean=mu[j], cov=sigma[j]).logpdf(y_con[i]))
            #eta[i,j] = np.exp(np.log(pi[j])+sp.stats.multivariate_normal(mean=mu[j], cov=sigma[j]).logpdf(y_con[i]))
        eta[i,:]=eta[i,:]/sum(eta[i,:])
        
    return eta

def M_STEP_pi(weights, eta):
    weights=weights.reshape((1,-1))
    return (weights@eta).reshape(-1)

def M_STEP_mu(y,y_prior,eta_1,eta_2,weights_1,weights_2, alph_conc, K_clusters, D):
        
    if alph_conc!=0:
        y_con=np.concatenate((y[0],y[1],y_prior),axis=0)
    else:
        y_con=np.concatenate((y[0],y[1]),axis=0)
        
    y_tot=np.concatenate((y_con,y_con),axis=0)
    w_tot=np.concatenate((weights_1,weights_2),axis=0)
    eta_tot=np.concatenate((eta_1,eta_2),axis=0)
    
    mu=np.zeros((K_clusters,D))
    for i in range(K_clusters):
        mu[i]=np.sum(y_tot*w_tot.reshape((-1,1))*eta_tot[:,i].reshape((-1,1)),axis=0)/np.sum(eta_tot[:,i])
    
    return mu

def M_STEP_sigma(y,y_prior,eta_1,eta_2,weights_1,weights_2, alph_conc, K_clusters, D, mu):
    
    
    if alph_conc!=0:
        y_con=np.concatenate((y[0],y[1],y_prior),axis=0)
    else:
        y_con=np.concatenate((y[0],y[1]),axis=0)
        
    y_tot=np.concatenate((y_con,y_con),axis=0)
    w_tot=np.concatenate((weights_1,weights_2),axis=0)
    eta_tot=np.concatenate((eta_1,eta_2),axis=0)
    
    sigma=np.zeros((K_clusters,D,D))
    for i in range(K_clusters):
        norm_y=(y_tot-mu[i])
        w_norm_y=norm_y*w_tot.reshape((-1,1))*eta_tot[:,i].reshape((-1,1))
        sigma[i]=w_norm_y.T@norm_y/np.sum(eta_tot[:,i])
    #print(sigma.shape)
    return sigma
    
    
    
    
    

In [7]:
#y, N_data, _, _, _=load_data(100,2)
#print(N_data)
#pi_init,mu_init,sigma_init = mgmm.init_params(y[0],N_data,3,2,10**(-7),100)
#kkk=np.array([np.diag(sigma_init[1]),np.diag(sigma_init[2])])
#kkk.reshape((1,-1)).shape

In [8]:
def main(K_set,B_postsamples,R_restarts): #B_postsamples is number of bootstrap samples, R_restarts is number of repeats in RR-NPL (set to 0 for FI-NPL)
    for n in range(n_iter):
        seed = 100+n

        np.random.seed(seed)
        y,N_data,K_clusters,D_data, K_set = load_data(seed,K_set)
        
        
        #prior settings
        alph_conc=0      #alph_concentration
        T_trunc = 500       #DP truncation
        tol = 1e-7
        max_iter = 100
        
        pi_bb_1=np.zeros((B_postsamples,K_clusters))
        pi_bb_2=np.zeros((B_postsamples,K_clusters))
        mu_bb=np.zeros((B_postsamples,K_clusters,D_data))
        sigma_bb=np.zeros((B_postsamples,K_clusters,D_data,D_data))
  

        start = time.time()
        weights_1, weights_2, y_prior=sample_weights_and_prior(B_postsamples, alph_conc, N_data, T_trunc, D_data, K_clusters, postsamples= None)
        for i in range(B_postsamples):
            pi_bb_1[i], pi_bb_2[i], mu_bb[i], sigma_bb[i] = hdp_gmm_joint(y,y_prior,weights_1[i],weights_2[i],N_data,K_clusters,D_data,alph_conc,tol=10**(-7),max_iter=50)
       
        end = time.time()

        print(end-start)

        #save file
        dict_bb = {'pi_1': pi_bb_1.tolist(),'sigma_1': sigma_bb.tolist(), 'mu_1': mu_bb.tolist(),'pi_2': pi_bb_2.tolist(),'sigma_2': sigma_bb.tolist(), 'mu_2': mu_bb.tolist(),'time': end-start}

        par_bb = pd.Series(data = dict_bb)

        if R_restarts ==0: 
            par_bb.to_pickle('./parameters/par_bb_sep_fi__B{}_seed{}'.format(B_postsamples,seed)) #uncomment for FI-NPL
        else:
            par_bb.to_pickle('./parameters/par_bb_sep_rr_rep{}_B{}_seed{}'.format(R_restarts,B_postsamples,seed))

if __name__=='__main__': 
    main(2,50,2)

/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: Conver

/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: Conver

/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: Conver

/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: Conver

/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
/Users/cuifuheng/Desktop/Joint的版本——TACC_using_HDP_simplified_updated！！ 2 2/experiments/Toy_GMM/npl/sk_base.py:277: Conver

51.10849595069885


In [ ]:




n_iter=2

def load_data(type,seed):
    #load test data
    gmm_data_test = np.load('./sim_data/gmm_data_test_{}_seed{}.npy'.format(type,seed),allow_pickle = True).item()

    #Extract parameters from data
    N_test = gmm_data_test['N'] #number of data in each dataset
    K_set = gmm_data_test['K_set'] #number of dataset
    K = gmm_data_test['K']
    D = gmm_data_test['D']
    y_test = gmm_data_test['y']#.reshape((2,N_test,D))
    
    return y_test,N_test,K,D,K_set


def load_posterior(method,type,seed,K):
    if method == 'RRNPL':
        #par = pd.read_pickle('./parameters/par_bb_{}_rr_rep{}_B{}_seed{}'.format(type,10,2000,seed))
        par = pd.read_pickle('./parameters/par_bb_{}_rr_rep{}_B{}_seed{}'.format(type,2,50,seed))
        pi_1 =np.array(par['pi_1'])
        mu_1 =np.array(par['mu_1'])
        sigma_1 = np.array(par[['sigma_1']][0])
        pi_2 =np.array(par['pi_2'])
        mu_2 =np.array(par['mu_2'])
        sigma_2 = np.array(par[['sigma_2']][0])
        time = par['time']

    elif method =='FINPL':
        #par = pd.read_pickle('./parameters/par_bb_{}_fi__B{}_seed{}'.format(type,2000,seed))
        par = pd.read_pickle('./parameters/par_bb_{}_fi__B{}_seed{}'.format(type,50,seed))
        pi_1 =np.array(par['pi_1'])
        mu_1 =np.array(par['mu_1'])
        sigma_1 = np.array(par[['sigma_1']][0])
        pi_2 =np.array(par['pi_2'])
        mu_2 =np.array(par['mu_2'])
        sigma_2 = np.array(par[['sigma_2']][0])
        time = par['time']

    elif method == 'NUTS':
        D = 1
        par = pd.read_pickle('./parameters/par_nuts_{}_seed{}'.format(type,seed))
        pi =par.iloc[:,3:K+3].values
        mu =par.iloc[:,3+K: 3+(K*(D+1))].values.reshape(2000,D,K).transpose(0,2,1)
        sigma = par.iloc[:,3+K*(D+1) :3+ K*(2*D+1)].values.reshape(2000,D,K).transpose(0,2,1)
        time = np.load('./parameters/time_nuts_{}_seed{}.npy'.format(type,seed),allow_pickle = True)

    elif method == 'ADVI':
        D = 1
        par = pd.read_pickle('./parameters/par_advi_{}_seed{}'.format(type,seed))
        pi =par.iloc[:,0:K].values
        mu =par.iloc[:,K: (K*(D+1))].values.reshape(2000,D,K).transpose(0,2,1)
        sigma = par.iloc[:,K*(D+1) : K*(2*D+1)].values.reshape(2000,D,K).transpose(0,2,1)
        time = np.load('./parameters/time_advi_{}_seed{}.npy'.format(type,seed),allow_pickle = True)


    return pi_1,mu_1,sigma_1,pi_2,mu_2,sigma_2,time

def eval(method,type):
    ll_test_1 = np.zeros(n_iter)
    ll_test_2 = np.zeros(n_iter)
    time = np.zeros(n_iter)
    for i in range(n_iter):
        seed = 100+i

        #Extract parameters from data
        y_test,N_test,K,D,K_set = load_data(type,seed)
        #print(np.shape(y_test))
        pi_1,mu_1,sigma_1,pi_2,mu_2,sigma_2,time[i]  = load_posterior(method,type,seed,K)
        #print(np.shape(sigma_1))
        #print(np.shape(mu_1))
        #ll_test_1[i] = gll.lppd(y_test[0,].reshape(-1,D),pi_1,mu_1, sigma_1,K)
        #ll_test_2[i] = gll.lppd(y_test[1,].reshape(-1,D),pi_2,mu_2, sigma_2,K)
        ll_test_1[i] = gll.lppd(y_test[0,],pi_1,mu_1, sigma_1,K)
        ll_test_2[i] = gll.lppd(y_test[1,],pi_2,mu_2, sigma_2,K)
                

    print('For {}, dataset {}'.format(method,type))
    print(np.mean(ll_test_1/N_test))
    print(np.std(ll_test_1/N_test))
    print(np.mean(ll_test_2/N_test))
    print(np.std(ll_test_2/N_test))

    print(np.mean(time))
    print(np.std(time))

def main():
    eval('RRNPL','sep')
    eval('FINPL','sep')
    #eval('NUTS','sep')
    #eval('ADVI','sep')

if __name__=='__main__':
    main()

In [9]:
par = pd.read_pickle('./parameters/par_bb_{}_rr_rep{}_B{}_seed{}'.format("sep",2,5,100))

In [10]:
        pi_1 =np.array(par['pi_1'])
        mu_1 =np.array(par['mu_1'])
        sigma_1 = np.array(par[['sigma_1']][0])
        pi_2 =np.array(par['pi_2'])
        mu_2 =np.array(par['mu_2'])
        sigma_2 = np.array(par[['sigma_2']][0])
        time = par['time']

In [16]:
np.mean(pi_2,axis=0)

array([0.24019611, 0.27048709, 0.4893168 ])